# Aula 5 - pandas

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Criar tabelas dinâmicas e fazer reshape do df (Melt, pivot, pivot_table)
- 2) Transformação de Dados (cut, qcut, get_dummies)
- 3) Utilidades Extras (multiindex to singleindex, combine_first)
_______

### Objetivos

Apresentar como criar tabelas dinâmicas, como fazer transformações em dados contínuos e categóricos e aprender como trabalhar com multiindex

____
____
____

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv("data/titanic.csv")

In [ ]:
df.drop(['PassengerId','Ticket','Name'],inplace=True,axis=1)

## Construindo uma Tabela Dinâmica usando Pandas

É hora de construir uma tabela dinâmica em Python usando a incrível biblioteca Pandas! Exploraremos as diferentes facetas de uma tabela dinâmica neste artigo e construiremos uma tabela dinâmica incrível e flexível a partir do zero.


    * pivot_table requer um dado e um parâmetro de índice
    * data é o dataframe do Pandas que você passa para a função
    * índice é o recurso que permite agrupar seus dados. O recurso de índice aparecerá como um índice na tabela resultante



In [ ]:
# index único
table = pd.pivot_table(data=df, index=['Sex'])
table

In [ ]:
# múltiplos indexes
table = pd.pivot_table(df, index=['Sex','Pclass'])
table

### Função de agregação
Por padrão o `.pivot_table()` utiliza o `np.mean()` como função de agragação, mas podemos utilizar diferentes funções de agregação para diferentes colunas. Para isso, precisamos de um dicionário como entrada para o parâmetro aggfunc com o nome da coluna como chave e a função agregada como o valor. <br>
Vamos criar uma pivot table calculando a média de 'Age' e a soma para o 'Survived':


In [ ]:
# diferentes funções de agregação
table = pd.pivot_table(df, 
                       index=['Sex','Pclass'], 
                       aggfunc={'Age':np.mean, 'Survived':np.sum})
table

Qual a diferença entre esse pivot_table e um groupby?

In [ ]:
df.groupby(['Sex','Pclass']).agg({'Age':"mean",'Survived':sum})

Para ficar mais parecido à tabela dinâmica do excel podemos indicar um dos index para ser visualizado como coluna e adicionar os totais de colunas e índices:

In [ ]:
table = pd.pivot_table(df,
                       index=['Sex'],
                       columns=['Pclass'],
                       values=['Survived'],
                       aggfunc=np.sum,
                       margins=1)
table

In [ ]:
table_prob = pd.pivot_table(df,
                       index=['Sex'],
                       columns=['Pclass'],
                       values=['Survived'],
                       aggfunc=np.mean,
                       margins=1)
table_prob

Formatando nossa saída

In [ ]:
(table_prob*100).style.format('{0:,.1f}%')

O `pd.pivot_table()` nos permite passar vários parâmetros úteis: <br>
pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True) <br>
Segue a mesma ideia de `pd.unstack()`.

## Desfazendo uma tabela dinâmica
Para fazer um unpivoting utilizamos o `pd.melt()`. Esse método é utilizado quando queremos que uma ou mais colunas se tornem colunas de identificadores. Segue a mesma ideia de `pd.stack()`. As colunas que vamos dissolver são definidas por `id_vars` e `value_vars`.

Parâmetros: <br>
pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)


   * __frame__: DataFrame <br>
   * __id_vars__: Colunas para serem usadas como categorias. São as colunas que você quer manter do jeito que estão. <br> 
   * __value_vars__: Colunas para o unpivot - para sairem do formato largo para longo (wide to long). Se não especificada, usa todas as colunas que não estão em id_vars. <br> 
   * __var_name__: Nome para a nova coluna de variáveis categóricas. <br>
   * __value_name__: Nome para ser utilizado na coluna de valores. <br>
   * __col_level__: Se as colunas são MultiIndex.<br>

Vamos simplificar nossa tabela de probabilidades eliminando a coluna e linha com os totais, resetando o index e eliminando o multi-index do nome das colunas renomeando-as.

In [ ]:
table_prob = table_prob.drop(('Survived', 'All'), axis=1).drop('All').reset_index()
table_prob.columns = ['Sex', 1, 2, 3]
table_prob

Observe o resultado final do nosso df ao utilizar o método `pd.melt()`:

In [ ]:
pd.melt(table_prob,
       id_vars=['Sex'])

Ele converteu as distintas colunas de Pclass em uma coluna com a categoria da classe e outra com seu valor. <br>
Para facilitar o entendimento das novas colunas podemos renomea-las:

In [ ]:
pd.melt(table_prob,
       id_vars=['Sex'],
       var_name='Class_melt',
       value_name='porc_of_survived')

## Transformação de dados

### pd.cut()
O método `pd.cut()` ordena os dados, separa em bins e computa qual grupo cada linha do df pertence. O `pd.cut()` escolherá os bins para serem espaçados uniformemente de acordo com os próprios valores e não com a frequência desses valores.  <br>
Ele é muito utilizado para transformar variáveis contínuas em categóricas. Por exemplo, podemos converter o valor númerico da idade em grupos de criança, jovem, adulto e idoso.
<br><br>
<a href='https://pandas.pydata.org/docs/reference/api/pandas.cut.html'>Parâmetros:</a> <br>
pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)

Ao informar quantidade de grupos o pd.cut() escolhe os bins com o mesmo tamanho de janela :


In [ ]:
df['cut_bins'] = pd.cut(df.Age, 4)
df.head()

Podemos passar o nome dos grupos e transformar a variável numérica diretamente em categórica


In [ ]:
df['cut_classes'] = pd.cut(df.Age, 4, labels=["jovens", "adultos", "meia-idade", "idosos"])
df.head()

In [ ]:
df.cut_bins.unique()

In [ ]:
df.cut_classes.value_counts()

In [ ]:
df.cut_bins.value_counts()

Também podemos passar uma lista com os valores de início e fim dos bins:

In [ ]:
pd.cut(df.Age, [0,20,60,80]).unique()

In [ ]:
df.Age.describe()

Repare que o ú

### pd.qcut()
O `pd.qcut()` é utilizado quando queremos discretizar nossos dados em quantis. Ao informar quantidade de grupos o `pd.qcut()` escolhe os bins tal que tenhamos a mesma quantidade de valores em cada grupo.

#### `pd.qcut()` x `pd.qcut()`
   * O comando `pd.cut()` cria **caixas equidistantes**, mas a **frequência** das amostras é **desigual** em cada caixa
   * O comando `pd.qcut()` cria **caixas de tamanhos desiguais**, mas a **frequência** das amostras é **igual** em cada caixa.

<br>
Parâmetros:<br>
pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')

In [ ]:
pd.cut(df.Age, 4).value_counts()

In [ ]:
pd.qcut(df.Age, 4).value_counts()

In [ ]:
pd.qcut(df.Age, 4).value_counts()/df.Age.notnull().sum()

<a href='https://towardsdatascience.com/discretisation-using-decision-trees-21910483fa4b'>Discretização utilizando decision trees</a>

### pd.get_dummies()

#### variáveis categóricas
Variáveis categóricas são aquelas que representam grupos ou classes dentro dos nossos dados. Elas podem ser de dois tipos:
* ordinais: possuem uma ordem que tem um sentido. Por exemplo, em rendimentos poderíamos ter: classe alta > classe média > classe baixa  
* nominais: não possuem uma ordem válida. Por exemplo: sexo e CEP.

<img src="variaveis_categoricas.jpeg" style="width: 500px">

Dummies são quaisquer variáveis cujos valores são 1 ou 0 para cada observação. O método `pd.get_dummies()` converte as variáveis categóricas em numéricas separando cada categoria em uma coluna única.
<br>
<br>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html">Parâmetros:</a> <br>
pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)

In [ ]:
pd.get_dummies(df, columns=['Sex', 'cut_classes'], drop_first=True)

In [ ]:
pd.get_dummies(pd.cut(df.Age, 4))

## Multi-index

In [ ]:
df.head()

Para setar indexes use o método `set_index()` indicando quais as colunas quer utilizar como uma lista.

In [ ]:
df_row_index = df.set_index(["Pclass", 'Sex'])
df_row_index

In [ ]:
df_row_index.index

Para acessar elementos:

In [ ]:
df_row_index.loc[(3, 'female')]

In [ ]:
df_row_index.reset_index(['Sex'])

### Multi-index nas colunas

In [ ]:
table

Acessando colunas:

In [ ]:
table.columns

Como acessar uma coluna:

In [ ]:
table[('Survived', 1)]

Slice usando multi-index

In [ ]:
table.loc[:, ('Survived', 1):('Survived', 3)]

Para obter o nome das colunas de cada nível hierárquico

In [ ]:
table.columns.get_level_values(0)

In [ ]:
table.columns.get_level_values(1)

In [ ]:
nivel_0 = table.columns.get_level_values(0)
nivel_1 = table.columns.get_level_values(1)

[j + '_' + str(nivel_1[i]) for i, j in enumerate(nivel_0)]

## Exercícios

1. Baixe os dados de consumo de bebidas por país do <a href="https://www.kaggle.com/justmarkham/alcohol-consumption-by-country">kaggle</a> faça uma análise das informações utilizando os métodos que você já conhece e depois responda:

In [ ]:
drinks = pd.read_csv("data/drinks.csv")
drinks

a. Encontre qual a bebida mais consumida em cada um dos países e a quantidade.

In [ ]:
df1 = drinks.set_index(['country','continent']).max(axis=1).reset_index()
df1

In [ ]:
df2 = drinks.set_index(['country','continent']).idxmax(axis=1).reset_index()
df2

In [ ]:
df2.merge(df1, on=['country', 'continent'])

b. Crie um df cujas bebidas estejam agrupadas em uma mesma coluna.

In [ ]:
drinks_melt = drinks.melt(id_vars=['country','continent'], value_name='volume', var_name='drink')
drinks_melt

c. Utilizando esse novo df, encontre qual a bebida mais consumida por país e a quantidade.

In [ ]:
drinks_melt.loc[drinks_melt.groupby('country').idxmax().volume]

2. Considere os dados de preço de fechamento e volume das ações que estão dentro de "data/stocks.csv". <br>
a. Escolha um método de python ensinado na aula de hoje para obter um dataframe cujas linhas são os códigos das ações e as colunas são as datas.

In [ ]:
stocks = pd.read_csv("data/stocks.csv")

In [ ]:
stocks

In [ ]:
stocks_pivot = stocks.pivot_table(values='Close', index='Symbol', columns='Date')
stocks_pivot

b. Com o df original, converta o código das ações para variáveis dummies.

In [ ]:
pd.get_dummies(stocks, columns=['Symbol'], drop_first=True)

3. Considere os dados do arquivo "german_credit.csv" que contem dados de empréstimos realizados por um banco.<br>
a. Encontre qual a média de empréstimo ("Credit Amount") obtidos considerando o propósito ("Purpose") do empréstimo nas linhas e o sexo ("Sex") nas colunas.

In [ ]:
gc = pd.read_csv('data/german_credit.csv')
gc.head()

In [ ]:
gc.pivot_table(index="Purpose", columns = ['Sex'] , values="Credit amount", aggfunc='mean')

In [ ]:
gc.groupby(['Purpose', 'Sex'])[['Credit amount']].mean()

b. Converta as variáveis categóricas em numéricas.

In [ ]:
gc.dtypes

In [ ]:
gc['Checking account'].unique()

In [ ]:
pd.get_dummies(gc, columns=['Saving accounts', 'Sex', 'Checking account', 'Purpose'], drop_first=True)

4. Considere o dataset (fake) com testes de aceleração para três carros distintos. Utilize um dos métodos ensinados em aula para criar uma única coluna com os valores das datas e outra com os valores das acelerações.

In [ ]:
s = 'Carro A'
x = 'Carro B'
three = 'Carro C'

s_data = [s, 2.5, 2.51, 2.54]
x_data = [x, 2.92, 2.91, 2.93]
three_data = [three, 3.33, 3.31, 3.35]

data = [s_data, x_data, three_data] 
car = pd.DataFrame(data, columns=['car_model', 'Sept 1 9am', 'Sept 1 10am', 'Sept 1 11am'])
car

In [ ]:
car_melt = car.melt(id_vars=['car_model'], var_name='date', value_name='0-60mph_in_seconds')
car_melt

## Referências:
pd.melt(): <br>
https://towardsdatascience.com/shape-tables-like-jelly-with-pandas-melt-and-pivot-f2e13e666d6 <br>
https://pub.towardsai.net/understanding-pandas-melt-pd-melt-362954f8c125